## Biased Review Detection

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data.tsv', sep='\t', encoding='utf-8',error_bad_lines=False,index_col=False)

b'Skipping line 4793: expected 10 fields, saw 11\nSkipping line 5508: expected 10 fields, saw 11\nSkipping line 6086: expected 10 fields, saw 11\nSkipping line 7489: expected 10 fields, saw 11\nSkipping line 8084: expected 10 fields, saw 11\nSkipping line 9580: expected 10 fields, saw 11\n'


In [3]:
#df.head()

## drop unwanted columns

In [4]:
df.drop(['DOC_ID'], axis=1)

,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...
...,...,...,...,...,...,...,...,...
20901,__label2__,4,Y,Shoes,B00BXYM8T8,"Madden Girl Women's Gettaw Pump,Red Patent,7.5...",wide width is great!,"I bought these for work. I have high arches, ..."
20902,__label2__,4,Y,Shoes,B0014C2ORK,"crocs Unisex Classic Clog,Khaki,6 US Men's / 8...",Love crocs!,Crocs are one of only two brands of shoes that...
20903,__label2__,5,Y,Shoes,B000EX8CCQ,Minnetonka Men's 703 Leather Laced Softsole Mo...,I love moccasins This fit like it was custom m...,I love moccasins This fit like it was custom ...
20904,__label2__,5,Y,Shoes,B00748YHVE,Ariat Womens Unbridled Fatbaby 9 B Powder Brown,"This fit well, comfortable, best investment",I wish these were a little more durable. I got...


## seggregate input and output(train and test) dataframes

In [5]:
x_input_df = df[df.columns.difference(['LABEL','DOC_ID'],sort=False)] #remove label and doc_id from the input dataframe 

In [6]:
y_label_df = df['LABEL']#create classifer data the label

In [7]:
y_label_df.head(n=2)

0    __label1__
1    __label1__
Name: LABEL, dtype: object

In [8]:
x_input_df.head(n=2)

,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...


## preprocess the text to removed unwanted punctuations

In [9]:
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')  
    return ReviewText

In [10]:
x_input_df['REVIEW_TEXT'] = preprocess(x_input_df['REVIEW_TEXT'])


## find the polarity and subjectivity of the review_text

In [11]:
from textblob import TextBlob
#df['polarity'] = df['REVIEW_TEXT'].apply(lambda text: sentiment_calc(text))
x_input_df['polarity'] = x_input_df['REVIEW_TEXT'].map(lambda text: TextBlob(text).sentiment.polarity)
x_input_df['subjectivity'] = x_input_df['REVIEW_TEXT'].map(lambda text: TextBlob(text).sentiment.subjectivity)

In [12]:
x_input_df['polarity'].head()               

0   -0.300000
1    0.219226
2    0.347143
3   -0.088542
4    0.145833
Name: polarity, dtype: float64

In [13]:
x_input_df.columns

Index(['RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY', 'PRODUCT_ID',
       'PRODUCT_TITLE', 'REVIEW_TITLE', 'REVIEW_TEXT', 'polarity',
       'subjectivity'],
      dtype='object')

## find word count of the review_text

In [14]:
x_input_df['word_count'] = x_input_df['REVIEW_TEXT'].apply(lambda x: len(x.split()))
x_input_df['word_count'].head()

0    23
1    69
2    52
3    44
4    66
Name: word_count, dtype: int64

## Convert verified purchase to binary

In [15]:
diag_map = {'N':1, 'Y':0}
# df_biased['VERIFIED_PURCHASE'] = df_biased['VERIFIED_PURCHASE'].map(diag_map)
# df_biased['VERIFIED_PURCHASE']
x_input_df['VERIFIED_PURCHASE'] = x_input_df['VERIFIED_PURCHASE'].map(diag_map)

## Converting product category from string to integer values

In [16]:

from numpy import array
data_array = array(x_input_df['PRODUCT_CATEGORY'] )

In [17]:
data_array

array(['PC', 'Wireless', 'Baby', ..., 'Shoes', 'Shoes', 'Shoes'],
      dtype=object)

In [18]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data_array)
print(integer_encoded)

[20 29  2 ... 22 22 22]


In [19]:
x_input_df['prod_cat_num'] = integer_encoded.tolist()

In [20]:
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)

In [21]:
# trial_df = x_input_df
# trial_df.PRODUCT_CATEGORY = pd.Categorical(trial_df.PRODUCT_CATEGORY)
# trial_df['code'] = trial_df.PRODUCT_CATEGORY.cat.codes

In [22]:
#x_input_df

In [23]:
# ?import nltk
#nltk.download('averaged_perceptron_tagger')

In [24]:
#!pip install textblob.download_corpora

In [25]:
from textblob import TextBlob

sLength = len(df['VERIFIED_PURCHASE'])
x_input_df['Nouns'] = pd.Series(0, index=x_input_df.index)
x_input_df['Verbs'] = pd.Series(0, index=x_input_df.index)
x_input_df['Adverbs'] = pd.Series(0, index=x_input_df.index)
x_input_df['Adjectives'] = pd.Series(0, index=x_input_df.index)

for index, row in x_input_df.iterrows():
    blob = TextBlob(row['REVIEW_TEXT'])
    for word,pos in blob.tags:
        if pos=='NN' or pos=='NNS' or pos=='NNP' or pos=='NNPS':
            x_input_df.at[index, 'Nouns']+= 1
        elif pos=='VP' or pos=='VB' or pos=='VBZ' or pos=='VBP' or pos=='VBD' or pos=='VBN' or pos=='VBG':
            x_input_df.at[index, 'Verbs']+= 1
        elif pos=='RB' or pos=='RBR' or pos=='RBS' or pos=='RP':
            x_input_df.at[index, 'Adverbs']+= 1
        elif pos=='JJ' or pos=='JJR' or pos=='JJS':
            x_input_df.at[index, 'Adjectives']+= 1

In [27]:
x_input_df

,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT,polarity,word_count,prod_cat_num,Nouns,Verbs,Adverbs,Adjectives
0,4,1,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav...",-0.300000,23,20,4,4,4,1
1,4,0,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...,0.219226,69,29,19,14,6,10
2,3,1,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...,0.347143,52,2,7,12,7,4
3,4,1,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...,-0.088542,44,18,8,10,4,3
4,4,1,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...,0.145833,66,3,13,10,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20901,4,0,Shoes,B00BXYM8T8,"Madden Girl Women's Gettaw Pump,Red Patent,7.5...",wide width is great!,"I bought these for work. I have high arches, ...",0.160000,21,22,5,4,2,2
20902,4,0,Shoes,B0014C2ORK,"crocs Unisex Classic Clog,Khaki,6 US Men's / 8...",Love crocs!,Crocs are one of only two brands of shoes that...,0.177083,25,22,10,2,1,1
20903,5,0,Shoes,B000EX8CCQ,Minnetonka Men's 703 Leather Laced Softsole Mo...,I love moccasins This fit like it was custom m...,I love moccasins This fit like it was custom ...,0.196049,56,22,13,12,5,7
20904,5,0,Shoes,B00748YHVE,Ariat Womens Unbridled Fatbaby 9 B Powder Brown,"This fit well, comfortable, best investment",I wish these were a little more durable. I got...,0.302500,58,22,12,12,1,5


In [1]:
#!pip install vaderSentiment

## Calculating Sentiment scores for the reviews

In [26]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [34]:
for review in x_input_df['REVIEW_TEXT']:
    vs = analyzer.polarity_scores(review)
#     x_input_df['negative','neutral','positive','compound'] = vs
    print(vs)
#     print("{:-<65} {}".format(review, str(vs)))

{'neg': 0.0, 'neu': 0.873, 'pos': 0.127, 'compound': 0.4939}
{'neg': 0.063, 'neu': 0.793, 'pos': 0.144, 'compound': 0.8019}
{'neg': 0.029, 'neu': 0.792, 'pos': 0.179, 'compound': 0.7897}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.877, 'pos': 0.123, 'compound': 0.7964}
{'neg': 0.034, 'neu': 0.733, 'pos': 0.234, 'compound': 0.8664}
{'neg': 0.0, 'neu': 0.815, 'pos': 0.185, 'compound': 0.7227}
{'neg': 0.0, 'neu': 0.663, 'pos': 0.337, 'compound': 0.9041}
{'neg': 0.0, 'neu': 0.685, 'pos': 0.315, 'compound': 0.8481}
{'neg': 0.049, 'neu': 0.785, 'pos': 0.166, 'compound': 0.7744}
{'neg': 0.241, 'neu': 0.759, 'pos': 0.0, 'compound': -0.6908}
{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.938}
{'neg': 0.024, 'neu': 0.71, 'pos': 0.266, 'compound': 0.9329}
{'neg': 0.242, 'neu': 0.57, 'pos': 0.188, 'compound': -0.5983}
{'neg': 0.077, 'neu': 0.782, 'pos': 0.14, 'compound': 0.7353}
{'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compound': 0.9638}
{'neg': 0.05, 'neu': 

{'neg': 0.052, 'neu': 0.762, 'pos': 0.186, 'compound': 0.9978}
{'neg': 0.022, 'neu': 0.937, 'pos': 0.041, 'compound': 0.296}
{'neg': 0.175, 'neu': 0.799, 'pos': 0.026, 'compound': -0.7474}
{'neg': 0.096, 'neu': 0.822, 'pos': 0.082, 'compound': -0.128}
{'neg': 0.021, 'neu': 0.843, 'pos': 0.136, 'compound': 0.8791}
{'neg': 0.0, 'neu': 0.489, 'pos': 0.511, 'compound': 0.94}
{'neg': 0.0, 'neu': 0.936, 'pos': 0.064, 'compound': 0.0258}
{'neg': 0.166, 'neu': 0.56, 'pos': 0.274, 'compound': 0.7157}
{'neg': 0.087, 'neu': 0.78, 'pos': 0.133, 'compound': 0.1543}
{'neg': 0.0, 'neu': 0.772, 'pos': 0.228, 'compound': 0.9349}
{'neg': 0.163, 'neu': 0.756, 'pos': 0.081, 'compound': -0.6907}
{'neg': 0.0, 'neu': 0.665, 'pos': 0.335, 'compound': 0.9523}
{'neg': 0.121, 'neu': 0.788, 'pos': 0.091, 'compound': -0.0516}
{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'compound': 0.5789}
{'neg': 0.0, 'neu': 0.649, 'pos': 0.351, 'compound': 0.8979}
{'neg': 0.093, 'neu': 0.745, 'pos': 0.161, 'compound': 0.6581}
{'neg'

{'neg': 0.07, 'neu': 0.73, 'pos': 0.2, 'compound': 0.9953}
{'neg': 0.0, 'neu': 0.688, 'pos': 0.312, 'compound': 0.9322}
{'neg': 0.307, 'neu': 0.622, 'pos': 0.071, 'compound': -0.8107}
{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.6124}
{'neg': 0.09, 'neu': 0.79, 'pos': 0.12, 'compound': 0.2725}
{'neg': 0.029, 'neu': 0.769, 'pos': 0.201, 'compound': 0.7506}
{'neg': 0.045, 'neu': 0.744, 'pos': 0.21, 'compound': 0.6249}
{'neg': 0.112, 'neu': 0.82, 'pos': 0.068, 'compound': -0.1306}
{'neg': 0.051, 'neu': 0.857, 'pos': 0.092, 'compound': 0.4574}
{'neg': 0.031, 'neu': 0.847, 'pos': 0.122, 'compound': 0.8829}
{'neg': 0.0, 'neu': 0.668, 'pos': 0.332, 'compound': 0.9678}
{'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'compound': 0.6908}
{'neg': 0.231, 'neu': 0.769, 'pos': 0.0, 'compound': -0.7579}
{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'compound': 0.6124}
{'neg': 0.0, 'neu': 0.802, 'pos': 0.198, 'compound': 0.9455}
{'neg': 0.124, 'neu': 0.876, 'pos': 0.0, 'compound': -0.872}
{'neg': 0.098, 'ne

{'neg': 0.164, 'neu': 0.708, 'pos': 0.128, 'compound': -0.9949}
{'neg': 0.0, 'neu': 0.689, 'pos': 0.311, 'compound': 0.9659}
{'neg': 0.0, 'neu': 0.558, 'pos': 0.442, 'compound': 0.969}
{'neg': 0.0, 'neu': 0.924, 'pos': 0.076, 'compound': 0.6486}
{'neg': 0.1, 'neu': 0.717, 'pos': 0.183, 'compound': 0.6355}
{'neg': 0.078, 'neu': 0.657, 'pos': 0.266, 'compound': 0.9118}
{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'compound': 0.9304}
{'neg': 0.126, 'neu': 0.874, 'pos': 0.0, 'compound': -0.7096}
{'neg': 0.057, 'neu': 0.82, 'pos': 0.122, 'compound': 0.4031}
{'neg': 0.072, 'neu': 0.755, 'pos': 0.174, 'compound': 0.7557}
{'neg': 0.024, 'neu': 0.976, 'pos': 0.0, 'compound': -0.3612}
{'neg': 0.118, 'neu': 0.678, 'pos': 0.204, 'compound': 0.8235}
{'neg': 0.041, 'neu': 0.603, 'pos': 0.357, 'compound': 0.9202}
{'neg': 0.091, 'neu': 0.796, 'pos': 0.113, 'compound': 0.2732}
{'neg': 0.05, 'neu': 0.753, 'pos': 0.196, 'compound': 0.7506}
{'neg': 0.19, 'neu': 0.81, 'pos': 0.0, 'compound': -0.8552}
{'neg': 0

{'neg': 0.065, 'neu': 0.86, 'pos': 0.075, 'compound': -0.5216}
{'neg': 0.063, 'neu': 0.55, 'pos': 0.387, 'compound': 0.9598}
{'neg': 0.063, 'neu': 0.937, 'pos': 0.0, 'compound': -0.1761}
{'neg': 0.028, 'neu': 0.823, 'pos': 0.149, 'compound': 0.8531}
{'neg': 0.034, 'neu': 0.91, 'pos': 0.055, 'compound': 0.1901}
{'neg': 0.052, 'neu': 0.634, 'pos': 0.315, 'compound': 0.9042}
{'neg': 0.137, 'neu': 0.752, 'pos': 0.111, 'compound': -0.1531}
{'neg': 0.0, 'neu': 0.919, 'pos': 0.081, 'compound': 0.3716}
{'neg': 0.01, 'neu': 0.765, 'pos': 0.226, 'compound': 0.989}
{'neg': 0.212, 'neu': 0.673, 'pos': 0.116, 'compound': -0.7814}
{'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'compound': 0.9502}
{'neg': 0.0, 'neu': 0.934, 'pos': 0.066, 'compound': 0.3919}
{'neg': 0.287, 'neu': 0.713, 'pos': 0.0, 'compound': -0.6167}
{'neg': 0.196, 'neu': 0.709, 'pos': 0.095, 'compound': -0.574}
{'neg': 0.0, 'neu': 0.646, 'pos': 0.354, 'compound': 0.8883}
{'neg': 0.132, 'neu': 0.868, 'pos': 0.0, 'compound': -0.6136}
{'neg

...

{'neg': 0.045, 'neu': 0.825, 'pos': 0.13, 'compound': 0.9668}
{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'compound': 0.9234}
{'neg': 0.008, 'neu': 0.926, 'pos': 0.065, 'compound': 0.9791}
{'neg': 0.057, 'neu': 0.76, 'pos': 0.183, 'compound': 0.8225}
{'neg': 0.02, 'neu': 0.848, 'pos': 0.133, 'compound': 0.7759}
{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.8934}
{'neg': 0.154, 'neu': 0.846, 'pos': 0.0, 'compound': -0.6256}
{'neg': 0.0, 'neu': 0.868, 'pos': 0.132, 'compound': 0.4404}
{'neg': 0.209, 'neu': 0.765, 'pos': 0.026, 'compound': -0.9413}
{'neg': 0.044, 'neu': 0.779, 'pos': 0.177, 'compound': 0.9307}
{'neg': 0.019, 'neu': 0.874, 'pos': 0.107, 'compound': 0.8306}
{'neg': 0.081, 'neu': 0.828, 'pos': 0.091, 'compound': 0.8749}
{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.8176}
{'neg': 0.048, 'neu': 0.823, 'pos': 0.129, 'compound': 0.8298}
{'neg': 0.069, 'neu': 0.753, 'pos': 0.178, 'compound': 0.7003}
{'neg': 0.041, 'neu': 0.729, 'pos': 0.23, 'compound': 0.8043}
{'ne

{'neg': 0.056, 'neu': 0.839, 'pos': 0.105, 'compound': 0.98}
{'neg': 0.0, 'neu': 0.7, 'pos': 0.3, 'compound': 0.9025}
{'neg': 0.0, 'neu': 0.755, 'pos': 0.245, 'compound': 0.9003}
{'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.9355}
{'neg': 0.102, 'neu': 0.8, 'pos': 0.098, 'compound': -0.0258}
{'neg': 0.047, 'neu': 0.928, 'pos': 0.025, 'compound': -0.3919}
{'neg': 0.197, 'neu': 0.67, 'pos': 0.133, 'compound': -0.7836}
{'neg': 0.139, 'neu': 0.861, 'pos': 0.0, 'compound': -0.7002}
{'neg': 0.126, 'neu': 0.767, 'pos': 0.108, 'compound': 0.1882}
{'neg': 0.036, 'neu': 0.772, 'pos': 0.192, 'compound': 0.9207}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.806, 'pos': 0.194, 'compound': 0.8917}
{'neg': 0.105, 'neu': 0.771, 'pos': 0.124, 'compound': 0.7789}
{'neg': 0.03, 'neu': 0.695, 'pos': 0.274, 'compound': 0.9107}
{'neg': 0.0, 'neu': 0.639, 'pos': 0.361, 'compound': 0.9731}
{'neg': 0.0, 'neu': 0.636, 'pos': 0.364, 'compound': 0.9436}
{'neg': 0.0, 'neu': 0

{'neg': 0.048, 'neu': 0.83, 'pos': 0.122, 'compound': 0.9711}
{'neg': 0.0, 'neu': 0.492, 'pos': 0.508, 'compound': 0.9474}
{'neg': 0.046, 'neu': 0.653, 'pos': 0.301, 'compound': 0.8931}
{'neg': 0.083, 'neu': 0.711, 'pos': 0.206, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'compound': 0.8705}
{'neg': 0.042, 'neu': 0.731, 'pos': 0.227, 'compound': 0.8625}
{'neg': 0.024, 'neu': 0.846, 'pos': 0.13, 'compound': 0.9315}
{'neg': 0.087, 'neu': 0.683, 'pos': 0.231, 'compound': 0.6021}
{'neg': 0.0, 'neu': 0.538, 'pos': 0.462, 'compound': 0.9292}
{'neg': 0.155, 'neu': 0.758, 'pos': 0.087, 'compound': -0.4019}
{'neg': 0.18, 'neu': 0.691, 'pos': 0.13, 'compound': -0.6696}
{'neg': 0.189, 'neu': 0.557, 'pos': 0.253, 'compound': 0.5095}
{'neg': 0.0, 'neu': 0.83, 'pos': 0.17, 'compound': 0.4012}
{'neg': 0.288, 'neu': 0.529, 'pos': 0.182, 'compound': -0.3818}
{'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'compound': 0.8625}
{'neg': 0.057, 'neu': 0.786, 'pos': 0.157, 'compound': 0.4939}
{'neg

{'neg': 0.171, 'neu': 0.728, 'pos': 0.101, 'compound': -0.987}
{'neg': 0.043, 'neu': 0.904, 'pos': 0.054, 'compound': 0.3682}
{'neg': 0.0, 'neu': 0.818, 'pos': 0.182, 'compound': 0.8986}
{'neg': 0.109, 'neu': 0.664, 'pos': 0.227, 'compound': 0.4588}
{'neg': 0.0, 'neu': 0.677, 'pos': 0.323, 'compound': 0.9081}
{'neg': 0.059, 'neu': 0.88, 'pos': 0.061, 'compound': 0.0258}
{'neg': 0.011, 'neu': 0.857, 'pos': 0.132, 'compound': 0.9387}
{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.9558}
{'neg': 0.0, 'neu': 0.79, 'pos': 0.21, 'compound': 0.75}
{'neg': 0.157, 'neu': 0.7, 'pos': 0.143, 'compound': 0.0292}
{'neg': 0.15, 'neu': 0.85, 'pos': 0.0, 'compound': -0.4585}
{'neg': 0.096, 'neu': 0.849, 'pos': 0.056, 'compound': -0.5563}
{'neg': 0.079, 'neu': 0.794, 'pos': 0.127, 'compound': 0.6045}
{'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compound': 0.856}
{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'compound': 0.8945}
{'neg': 0.046, 'neu': 0.817, 'pos': 0.138, 'compound': 0.8484}
{'neg': 0.057, 'ne

{'neg': 0.067, 'neu': 0.815, 'pos': 0.118, 'compound': 0.9857}
{'neg': 0.0, 'neu': 0.898, 'pos': 0.102, 'compound': 0.3612}
{'neg': 0.14, 'neu': 0.777, 'pos': 0.083, 'compound': -0.4411}
{'neg': 0.0, 'neu': 0.767, 'pos': 0.233, 'compound': 0.9258}
{'neg': 0.073, 'neu': 0.927, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.072, 'neu': 0.848, 'pos': 0.08, 'compound': 0.0772}
{'neg': 0.085, 'neu': 0.784, 'pos': 0.132, 'compound': 0.324}
{'neg': 0.0, 'neu': 0.565, 'pos': 0.435, 'compound': 0.9062}
{'neg': 0.0, 'neu': 0.927, 'pos': 0.073, 'compound': 0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.035, 'neu': 0.858, 'pos': 0.107, 'compound': 0.901}
{'neg': 0.0, 'neu': 0.695, 'pos': 0.305, 'compound': 0.937}
{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.8478}
{'neg': 0.0, 'neu': 0.46, 'pos': 0.54, 'compound': 0.9558}
{'neg': 0.0, 'neu': 0.795, 'pos': 0.205, 'compound': 0.7351}
{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'compound': 0.9475}
{'neg': 0.0, 'neu': 0.871, '

In [27]:
final_list = []
negative = []
neutral = []
positive = []
compound = []

for review in x_input_df['REVIEW_TEXT']:
    vs = analyzer.polarity_scores(review)
    final_list.append(list(vs.values()))


In [28]:
for value in final_list:
    negative.append(value[0])
    neutral.append(value[1])
    positive.append(value[2])
    compound.append(value[3])

In [29]:
# import numpy as np
x_input_df['negative'] = negative
x_input_df['neutral'] = neutral
x_input_df['positive'] = positive
x_input_df['compound'] = compound
# df = pd.DataFrame(np.array(final_list).reshape(20906,4), columns = list("abcd"))
# print (df)

In [30]:
x_input_df

,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT,polarity,subjectivity,word_count,prod_cat_num,Nouns,Verbs,Adverbs,Adjectives,negative,neutral,positive,compound
0,4,1,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav...",-0.300000,0.400000,23,20,4,4,4,1,0.000,0.873,0.127,0.4939
1,4,0,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...,0.219226,0.385320,69,29,19,14,6,10,0.063,0.793,0.144,0.8019
2,3,1,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...,0.347143,0.600000,52,2,7,12,7,4,0.029,0.792,0.179,0.7897
3,4,1,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...,-0.088542,0.441667,44,18,8,10,4,3,0.000,1.000,0.000,0.0000
4,4,1,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...,0.145833,0.525000,66,3,13,10,3,6,0.000,0.877,0.123,0.7964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20901,4,0,Shoes,B00BXYM8T8,"Madden Girl Women's Gettaw Pump,Red Patent,7.5...",wide width is great!,"I bought these for work. I have high arches, ...",0.160000,0.540000,21,22,5,4,2,2,0.000,0.863,0.137,0.4019
20902,4,0,Shoes,B0014C2ORK,"crocs Unisex Classic Clog,Khaki,6 US Men's / 8...",Love crocs!,Crocs are one of only two brands of shoes that...,0.177083,0.441667,25,22,10,2,1,1,0.000,0.828,0.172,0.6988
20903,5,0,Shoes,B000EX8CCQ,Minnetonka Men's 703 Leather Laced Softsole Mo...,I love moccasins This fit like it was custom m...,I love moccasins This fit like it was custom ...,0.196049,0.428395,56,22,13,12,5,7,0.019,0.730,0.251,0.9390
20904,5,0,Shoes,B00748YHVE,Ariat Womens Unbridled Fatbaby 9 B Powder Brown,"This fit well, comfortable, best investment",I wish these were a little more durable. I got...,0.302500,0.460000,58,22,12,12,1,5,0.123,0.720,0.157,0.4767


## Calculating modality of the sentences

In [31]:
# !pip install pattern
from pattern.en import parse, Sentence, parse
from pattern.en import modality,mood

In [34]:
lst = [round(modality(review),2) for review in x_input_df['REVIEW_TEXT']]
#print(lst)

In [35]:
x_input_df['modality'] = lst


In [115]:
# !pip install nltk
# from sklearn.preprocessing import StandardScaler
# x_input_df = StandardScaler().fit_transform(x_input_df)

## Implementing PCA to identify principal components

In [69]:
features = ['VERIFIED_PURCHASE','polarity','word_count','Nouns','Verbs','Adjectives','Adverbs'];

x_input_features = x_input_df[features]

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x_input_features)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal_component_1', 'principal_component_2'])

In [70]:
principalDf

,principal_component_1,principal_component_2
0,-49.527355,0.586954
1,0.431123,0.446954
2,-19.804188,-6.111721
3,-27.577921,-2.305219
4,-5.387622,-3.229914
...,...,...
20901,-51.146161,2.313426
20902,-46.383632,6.466148
20903,-14.151489,-1.265537
20904,-13.030206,-2.200229


## Split dataset into train and test

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

# features = ['VERIFIED_PURCHASE','polarity','word_count','RATING','prod_cat_num'];
# x = x_input_df[features]

X_train, X_test, y_train, y_test = train_test_split(x_input_features, y_label_df, test_size=0.25)
# y_train.isna().sum()


## Implementing Logistic Regression

In [131]:
model = LogisticRegression()
model.fit(X_train, y_train)
predicted_classes = model.predict(X_test)
accuracy = accuracy_score(y_test,predicted_classes)
print(accuracy)

0.7903194949301703


In [132]:
print(confusion_matrix(y_test,predicted_classes))

[[1791  754]
 [ 342 2340]]


In [133]:
print(classification_report(y_test,predicted_classes))

              precision    recall  f1-score   support

  __label1__       0.84      0.70      0.77      2545
  __label2__       0.76      0.87      0.81      2682

    accuracy                           0.79      5227
   macro avg       0.80      0.79      0.79      5227
weighted avg       0.80      0.79      0.79      5227



## Implementing SVM

In [134]:
from sklearn.svm import SVC
clf_SVM = SVC(gamma='auto')
clf_SVM.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [135]:
predicted_SVM=clf_SVM.predict(X_test)
accuracy_SVM = accuracy_score(y_test,predicted_SVM)
accuracy_SVM

0.7338817677444041

In [136]:
print(confusion_matrix(y_test,predicted_SVM))

[[1533 1012]
 [ 379 2303]]


In [137]:
print(classification_report(y_test,predicted_SVM))

              precision    recall  f1-score   support

  __label1__       0.80      0.60      0.69      2545
  __label2__       0.69      0.86      0.77      2682

    accuracy                           0.73      5227
   macro avg       0.75      0.73      0.73      5227
weighted avg       0.75      0.73      0.73      5227



## Implementing decision tree algorithm

In [48]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [49]:
# def train_using_gini(X_train, X_test, y_train): 
  
#     # Creating the classifier object 
#     clf_gini = DecisionTreeClassifier(criterion = "gini", 
#             random_state = 100,max_depth=3, min_samples_leaf=5) 
  
#     # Performing training 
#     clf_gini.fit(X_train, y_train) 
#     return clf_gini 

In [50]:
def train_using_entropy(X_train, X_test, y_train): 
  
    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 100, 
            max_depth = 3, min_samples_leaf = 5) 
  
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy 

In [178]:
# features1 = ['VERIFIED_PURCHASE','polarity','word_count','Nouns','Verbs','Adjectives','Adverbs','modality','negative','positive'];
# x_input_for_dtree = x_input_df[features1]
# x_train, x_test, Y_train, Y_test = train_test_split(x_input_for_dtree, y_label_df, test_size=0.25)

In [51]:
# gini_res = train_using_gini(X_train, X_test, y_train)
# entropy_res = train_using_entropy(X_train,X_test,y_train)

In [52]:
# y_pred_gini = gini_res.predict(X_test)
# accuracy_score( y_test,y_pred_gini)

0.7968241821312416

In [101]:
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y_test,y_pred_gini))

[[1097 1481]
 [1127 1522]]


In [100]:
# print(classification_report(y_test,y_pred_gini))

              precision    recall  f1-score   support

  __label1__       0.49      0.43      0.46      2578
  __label2__       0.51      0.57      0.54      2649

    accuracy                           0.50      5227
   macro avg       0.50      0.50      0.50      5227
weighted avg       0.50      0.50      0.50      5227



In [55]:
y_pred_entropy = entropy_res.predict(X_test)
accuracy_score(y_test, y_pred_entropy)

0.7968241821312416

In [102]:
print(confusion_matrix(y_test,y_pred_entropy))

[[1097 1481]
 [1127 1522]]


In [103]:
print(classification_report(y_test,y_pred_entropy))

              precision    recall  f1-score   support

  __label1__       0.49      0.43      0.46      2578
  __label2__       0.51      0.57      0.54      2649

    accuracy                           0.50      5227
   macro avg       0.50      0.50      0.50      5227
weighted avg       0.50      0.50      0.50      5227



In [58]:
# from sklearn import tree

# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X_train, y_train)
# predict_dtree = clf.predict(X_test)
# accuracy_dtree = accuracy_score(y_test,predict_dtree)
# print(accuracy_dtree)

0.6927491869140998


## ensemble methods - gradient boosting

In [115]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(X_train, y_train)
pred_gb = model.predict(X_test)
accuracy_gb = accuracy_score(y_test,pred_gb)
accuracy_gb

0.7933805242012627

In [120]:
print(confusion_matrix(y_test,pred_gb))

[[1772  773]
 [ 307 2375]]


In [121]:
print(classification_report(y_test,pred_gb))

              precision    recall  f1-score   support

  __label1__       0.85      0.70      0.77      2545
  __label2__       0.75      0.89      0.81      2682

    accuracy                           0.79      5227
   macro avg       0.80      0.79      0.79      5227
weighted avg       0.80      0.79      0.79      5227



## ensemble methods - xGBoost(extreme Gradient Boosting)

In [119]:
#!pip install xgboost
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(X_train, y_train)
pred_xgb = model.predict(X_test)
accuracy_xgb = accuracy_score(y_test,pred_xgb)
accuracy_xgb

0.7929978955423761

In [122]:
print(confusion_matrix(y_test,pred_xgb))

[[1774  771]
 [ 311 2371]]


In [123]:
print(classification_report(y_test,pred_xgb))

              precision    recall  f1-score   support

  __label1__       0.85      0.70      0.77      2545
  __label2__       0.75      0.88      0.81      2682

    accuracy                           0.79      5227
   macro avg       0.80      0.79      0.79      5227
weighted avg       0.80      0.79      0.79      5227



## ensemble methods - adaBoost (Adaptive boosting)

In [124]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(X_train, y_train)
pred_adb = model.predict(X_test)
accuracy_adb = accuracy_score(y_test,pred_adb)
accuracy_adb

0.7920413238951598

In [125]:
print(confusion_matrix(y_test,pred_adb))

[[1792  753]
 [ 334 2348]]


In [126]:
print(classification_report(y_test,pred_adb))

              precision    recall  f1-score   support

  __label1__       0.84      0.70      0.77      2545
  __label2__       0.76      0.88      0.81      2682

    accuracy                           0.79      5227
   macro avg       0.80      0.79      0.79      5227
weighted avg       0.80      0.79      0.79      5227



## bagging - meta estimator 

In [127]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(X_train, y_train)
pred_bag = model.predict(X_test)
accuracy_bag = accuracy_score(y_test,pred_bag)
accuracy_bag

0.7367514826860532

In [128]:
print(confusion_matrix(y_test,pred_bag))

[[1870  675]
 [ 701 1981]]


In [129]:
print(classification_report(y_test,pred_bag))

              precision    recall  f1-score   support

  __label1__       0.73      0.73      0.73      2545
  __label2__       0.75      0.74      0.74      2682

    accuracy                           0.74      5227
   macro avg       0.74      0.74      0.74      5227
weighted avg       0.74      0.74      0.74      5227



In [130]:
# import seaborn as sns

# myDictionary = {'LR':0.794,'SVM':0.740,'DecTree':0.797,'GB':0.797,'XGB':0.797,'AB':0.794,'BME':0.753}
# fig = plt.figure(figsize=(16,10), dpi= 80)
# fig.suptitle('Algorithm comparison plot', fontsize=20, fontweight='bold')
# width = 0.25
# ax = fig.add_subplot(111)
# # ax.set_title('axes title')
# ax.set_xlabel('Algorithms')
# ax.set_ylabel('Accuracy')
# plt.bar(myDictionary.keys(), myDictionary.values(),width, color='b')
# #sns.kdeplotplot(myDict)